In [1]:
#from config import (census_key, gkey)
import numpy as np
import pandas as pd
import requests
import zillow
# import pprint as pp
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [2]:
connection_string = "postgres:apartment@homeslice.cjnrjw08kldx.us-east-2.rds.amazonaws.com:5432/HomeSliceDB"
engine = create_engine(f'postgresql://{connection_string}')

In [3]:
#Zillow Data Cleaning

data = engine.execute('SELECT * FROM zillow').fetchall()

In [4]:

df = pd.DataFrame(data, columns = ['price_zestimate','currency','price_last_updated','amount_chng_30days',
                                   'valuation_range_low','valuation_range_high','street','zipcode','city',
                                   'state','latitude','longitude','risk_adj_val','max_cash_limit','time_stamp'])
df.head()

,price_zestimate,currency,price_last_updated,amount_chng_30days,valuation_range_low,valuation_range_high,street,zipcode,city,state,latitude,longitude,risk_adj_val,max_cash_limit,time_stamp
0,195321.0,USD,04/07/2020,3084.0,177742.0,218760.0,501 Shumaker Dr,15146,Monroeville,PA,40.456694,-79.763917,166022.85,49806.855,NaT
1,199524.0,USD,04/08/2020,7174.0,183562.0,223467.0,501 Shumaker Dr,15146,Monroeville,PA,40.456694,-79.763917,169595.40,50878.620,NaT
2,169555.0,USD,04/08/2020,3881.0,152600.0,186510.0,504 Shumaker Dr,15146,Monroeville,PA,40.456537,-79.762814,144121.75,43236.525,NaT
3,405867.0,USD,04/08/2020,-242.0,369339.0,438336.0,1901 Oxmead Rd,08016,Burlington,NJ,40.040357,-74.802437,344986.95,103496.085,NaT
4,494180.0,USD,04/08/2020,-316.0,449704.0,553482.0,317 E 8th Ave,08260,Wildwood,NJ,39.00134,-74.794258,420053.00,126015.900,NaT


In [5]:
tableau_df = df.drop_duplicates(subset=['street'], keep='first')
tableau_df

,price_zestimate,currency,price_last_updated,amount_chng_30days,valuation_range_low,valuation_range_high,street,zipcode,city,state,latitude,longitude,risk_adj_val,max_cash_limit,time_stamp
0,195321.0,USD,04/07/2020,3084.0,177742.0,218760.0,501 Shumaker Dr,15146,Monroeville,PA,40.456694,-79.763917,166022.85,49806.855,NaT
2,169555.0,USD,04/08/2020,3881.0,152600.0,186510.0,504 Shumaker Dr,15146,Monroeville,PA,40.456537,-79.762814,144121.75,43236.525,NaT
3,405867.0,USD,04/08/2020,-242.0,369339.0,438336.0,1901 Oxmead Rd,08016,Burlington,NJ,40.040357,-74.802437,344986.95,103496.085,NaT
4,494180.0,USD,04/08/2020,-316.0,449704.0,553482.0,317 E 8th Ave,08260,Wildwood,NJ,39.00134,-74.794258,420053.00,126015.900,NaT
9,548586.0,USD,04/08/2020,6406.0,488242.0,614416.0,502 Shumaker Dr,15146,Monroeville,PA,40.456284,-79.76307,466298.10,139889.430,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,1100184.0,USD,04/12/2020,64953.0,1045175.0,1155193.0,382 Central Park W APT 2B,10025,New York,NY,40.79367,-73.965556,935156.40,280546.920,2020-04-14 08:54:32.537463
219,1780210.0,USD,04/12/2020,-716.0,1691200.0,1869220.0,308 W 97th St APT 34,10025,New York,NY,40.795855,-73.9729,1513178.50,453953.550,2020-04-14 08:55:50.250260
220,740805.0,USD,04/12/2020,-492.0,703765.0,777845.0,26 W 97th St APT 3B,10025,New York,NY,40.792503,-73.965175,629684.25,188905.275,2020-04-14 08:56:16.935283
221,4124833.0,USD,04/12/2020,-309733.0,3877343.0,4496068.0,130 W 95th St,10025,New york,NY,40.792498,-73.969338,3506108.05,1051832.415,2020-04-14 08:56:37.352519


In [6]:
#User Data Table Export to CSV

tableau_df.to_csv('Tableau_data/tableau_user_input_data.csv', index=False)  

In [7]:
#Sales Price Historical Data Cleaning

session = Session(engine)
inspector = inspect(engine)
inspector.get_table_names()

['zillow', 'sales_price', 'user_data']

In [8]:
# Extract column names from desired table

columns = inspector.get_columns('sales_price')

names = []

for c in columns:
    names.append(c['name'])

In [9]:
#Sales Price Historical Data Cleaning

sales_data = engine.execute('SELECT * FROM sales_price').fetchall()
sales_df = pd.DataFrame(sales_data,columns=names)
sales_df.head()

,zip_code,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,...,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,percent_change,equity_risk
0,10025,269259.83,264101.03,257829.16,253550.8,250369.45,248516.75,246508.74,243987.72,240695.55,...,1218400.0,1024700.0,1031100.0,837800.0,965500.0,896100.0,873100.0,852000.0,2.164230,True
1,60657,370100.00,391300.00,388700.00,382200.0,381300.00,378500.00,364600.00,362300.00,357100.00,...,371300.0,398000.0,418200.0,424500.0,425700.0,411900.0,400500.0,390300.0,0.054580,True
2,10023,269259.83,264101.03,257829.16,253550.8,250369.45,248516.75,246508.74,243987.72,240695.55,...,1127200.0,1227900.0,1142000.0,1145000.0,1049600.0,1014900.0,1044600.0,1142200.0,3.241999,True
3,77494,232000.00,240900.00,243000.00,255500.0,253400.00,259000.00,253200.00,238900.00,228000.00,...,331900.0,328300.0,320900.0,315100.0,317500.0,315200.0,319000.0,313000.0,0.349138,True
4,60614,269259.83,264101.03,257829.16,253550.8,250369.45,248516.75,246508.74,243987.72,240695.55,...,510300.0,490900.0,497300.0,468100.0,489700.0,484000.0,537700.0,566600.0,1.104287,True


In [10]:
#Sales Price Historical Sales Data by Zip

# sales_df.to_csv('Tableau_data/tableau_salesprice_hist_zip.csv', index=False)

In [11]:
sales_df.columns = sales_df.columns.str.split('-').str[0]
sales_df.set_index(['zip_code'])
sales_df.head()

,zip_code,2008,2008,2008,2008,2008,2008,2008,2008,2008,...,2019,2019,2019,2019,2019,2019,2019,2019,percent_change,equity_risk
0,10025,269259.83,264101.03,257829.16,253550.8,250369.45,248516.75,246508.74,243987.72,240695.55,...,1218400.0,1024700.0,1031100.0,837800.0,965500.0,896100.0,873100.0,852000.0,2.164230,True
1,60657,370100.00,391300.00,388700.00,382200.0,381300.00,378500.00,364600.00,362300.00,357100.00,...,371300.0,398000.0,418200.0,424500.0,425700.0,411900.0,400500.0,390300.0,0.054580,True
2,10023,269259.83,264101.03,257829.16,253550.8,250369.45,248516.75,246508.74,243987.72,240695.55,...,1127200.0,1227900.0,1142000.0,1145000.0,1049600.0,1014900.0,1044600.0,1142200.0,3.241999,True
3,77494,232000.00,240900.00,243000.00,255500.0,253400.00,259000.00,253200.00,238900.00,228000.00,...,331900.0,328300.0,320900.0,315100.0,317500.0,315200.0,319000.0,313000.0,0.349138,True
4,60614,269259.83,264101.03,257829.16,253550.8,250369.45,248516.75,246508.74,243987.72,240695.55,...,510300.0,490900.0,497300.0,468100.0,489700.0,484000.0,537700.0,566600.0,1.104287,True


In [12]:
sales_df['2008_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2008')]].mean(axis=1)
sales_df['2009_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2009')]].mean(axis=1)
sales_df['2010_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2010')]].mean(axis=1)
sales_df['2011_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2011')]].mean(axis=1)
sales_df['2012_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2012')]].mean(axis=1)
sales_df['2013_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2013')]].mean(axis=1)
sales_df['2014_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2014')]].mean(axis=1)
sales_df['2015_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2015')]].mean(axis=1)
sales_df['2016_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2016')]].mean(axis=1)
sales_df['2017_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2017')]].mean(axis=1)
sales_df['2018_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2018')]].mean(axis=1)
sales_df['2019_avg_salespr']= sales_df.loc[:, [x for x in sales_df.columns if x.startswith('2019')]].mean(axis=1)
sales_df.drop(['2008', '2009','2010','2011','2012',
                                      '2013','2014','2015','2016','2017','2018','2019'], axis=1, inplace=True)

sales_df.to_csv('Tableau_data/tableau_avgsalespr_yr.csv', index=False)

In [13]:
#NEED TO FIX ALL BELOW


# raw_dates_sales = sales_df.drop(columns=['percent_change', 'equity_risk'])
# raw_dates_sales.head()

In [14]:
# zip_sales = raw_dates_sales.set_index(['zip_code'])

In [15]:
# sales_transposed = zip_sales.T # or df1.transpose()
# sales_transposed.tail()

In [16]:
# sales_by_date = sales_transposed.rename_axis('date').reset_index()
# sales_by_date['date'] = pd.to_datetime(sales_by_date['date'])
# sales_by_date.tail()

In [17]:
# #Sales Price Historical Sales Data by Date

# sales_by_date.to_csv('Tableau_data/tableau_salesprice_hist_date.csv', index=False)